In [23]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import rasterio
import pandas as pd
from damage import features
from damage.data import Raster

def crop_annotation_to_image_dimensions(annotation_data, dimensions):
    mask = (annotation_data['row'] < dimensions['height'])\
        & (annotation_data['row'] >= 0)\
        & (annotation_data['column'] < dimensions['width'])\
        & (annotation_data['column'] >= 0)
    return annotation_data.loc[mask]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
annotations = gpd.read_file('../data/annotations/4_Damage_Sites_Daraa_CDA.shp')
annotations['longitude'] = annotations['geometry'].apply(lambda x: x.x)
annotations['latitude'] = annotations['geometry'].apply(lambda x: x.y)
raster_daraa = rasterio.open('../data/city_rasters/daraa_2011_10_17_zoom_19.tif')
columns, rows = raster_daraa.index(annotations['longitude'], annotations['latitude'])
annotations['column'] = columns
annotations['row'] = rows
annotations = crop_annotation_to_image_dimensions(annotations, {'width': raster_daraa.width, 'height': raster_daraa.height})
no_analysis = gpd.read_file('../data/polygons/5_No_Analysis_Areas_Daraa.shp')
populated_areas = gpd.read_file('../data/polygons/populated_areas.shp')
populated_areas['NAME_EN'] = populated_areas['NAME_EN'].str.lower()

In [15]:
data = {
    'raster_daraa_2011_10_17_zoom_19.tif': Raster('../data/city_rasters/daraa_2011_10_17_zoom_19.tif'),
    'no_analysis': no_analysis,
    'populated_areas': populated_areas,
    'annotations': annotations,
}
annotation_preprocessor = features.AnnotationPreprocessor()
data = annotation_preprocessor.transform(data)

tile_size = 64
stride = tile_size
raster_splitter = features.RasterSplitter(tile_size=tile_size, stride=stride)
raster_split = raster_splitter.transform(data)

> /Users/yc00031/repos/syria/damage/features/raster_splitter.py(43)_split_raster_data()
     42             import ipdb; ipdb.set_trace()
---> 43             for w in tqdm(range(self.tile_size//2, (raster.width - self.tile_size//2), self.stride)):
     44                 for h in range(self.tile_size//2, (raster.height - self.tile_size//2), self.stride):



ipdb>  c


100%|██████████| 207/207 [00:30<00:00,  6.76it/s]


> /Users/yc00031/repos/syria/damage/features/raster_splitter.py(20)transform()
     19         import ipdb; ipdb.set_trace()
---> 20         tiles['location_index'] = geo_location_index(tiles['longitude'], tiles['latitude'],
     21                                                      grid_size=self.grid_size)



ipdb>  c


In [18]:
raster_split.head()

,city,date,image,latitude,longitude,split_id,location_index
0,daraa,2011-10-17,"[[[90.0, 61.0, 49.0], [90.0, 65.0, 58.0], [99....",32.642610,36.073439,992-3424,457420059100
1,daraa,2011-10-17,"[[[99.0, 65.0, 58.0], [99.0, 65.0, 58.0], [107...",32.642610,36.073783,1056-3424,457421203958
2,daraa,2011-10-17,"[[[90.0, 65.0, 58.0], [107.0, 77.0, 66.0], [11...",32.642610,36.074126,1120-3424,457422348816
3,daraa,2011-10-17,"[[[74.0, 53.0, 41.0], [99.0, 69.0, 58.0], [123...",32.637803,36.074126,1120-4320,457422348804
4,daraa,2011-10-17,"[[[99.0, 69.0, 66.0], [99.0, 65.0, 58.0], [99....",32.643640,36.074469,1184-3232,457424638534


In [20]:
annotations.head()

,SiteID,SensDt,SensorID,ConfID,DmgCls,SensDt_2,SensID_2,ConfID_2,DmgCls_2,DmgSts_2,...,Notes,StlmtNme,Neighbrhd,EventCode,geometry,longitude,latitude,column,row,location_index
0,Mosque,2013-09-07,Worldview-2,Very High,Severe Damage,2014-05-01,Worldview-2,Very High,Severe Damage,No change,...,Omari Mosque (destroyed minaret),Daraa,Dar'a al-Balad,CE20130604SYR,POINT (36.10085583300008 32.61267652900005),36.100856,32.612677,9004,6103,444864412648
1,Building (General / Default),2013-09-07,Worldview-2,Very High,Severe Damage,2014-05-01,Worldview-2,Very High,Severe Damage,No change,...,<Null>,Daraa,Dar'a al-Balad,CE20130604SYR,POINT (36.10088763300007 32.61285126300004),36.100888,32.612851,8971,6109,444864412648
2,Building (General / Default),2013-09-07,Worldview-2,Medium,Moderate Damage,2014-05-01,Worldview-2,Medium,Moderate Damage,No change,...,<Null>,Daraa,Dar'a al-Balad,CE20130604SYR,POINT (36.10061228100005 32.61282301800009),36.100612,32.612823,8977,6057,444864412647
3,Building (General / Default),2013-09-07,Worldview-2,Very High,Severe Damage,2014-05-01,Worldview-2,Very High,Severe Damage,No change,...,<Null>,Daraa,Dar'a al-Balad,CE20130604SYR,POINT (36.10040312800004 32.61289420500009),36.100403,32.612894,8963,6018,444864412647
4,Building (General / Default),2013-09-07,Worldview-2,Uncertain,Moderate Damage,2014-05-01,Worldview-2,Uncertain,Moderate Damage,No change,...,<Null>,Daraa,Dar'a al-Balad,CE20130604SYR,POINT (36.10014904700006 32.61300812300004),36.100149,32.613008,8942,5971,444865557504


In [25]:
annotations.columns

Index(['SiteID', 'SensDt', 'SensorID', 'ConfID', 'DmgCls', 'SensDt_2',
       'SensID_2', 'ConfID_2', 'DmgCls_2', 'DmgSts_2', 'SensDt_3', 'SensID_3',
       'ConfID_3', 'DmgCls_3', 'DmgSts_3', 'SensDt_4', 'SensID_4', 'ConfID_4',
       'DmgCls_4', 'DmgSts_4', 'GrpDmgCls', 'FldValid', 'Notes', 'StlmtNme',
       'Neighbrhd', 'EventCode', 'geometry', 'longitude', 'latitude', 'column',
       'row', 'location_index'],
      dtype='object')

In [24]:
pd.merge(raster_split, annotations, on=['city', 'location_index', 'date']).head(1)

KeyError: 'city'

In [27]:
a = pd.DataFrame({
    'i0': ['a', 'a'],
    'i1': ['b', 'b'],
    'col_a': [0, 1],
}).set_index(['i0', 'i1'])
b = pd.DataFrame({
    'i0': ['a', 'a'],
    'i1': ['b', 'c'],
    'col_b': [0, 1],
}).set_index(['i0', 'i1'])

In [28]:
pd.merge(a, b, right_index=True, left_index=True)

col_a  col_b
i0 i1              
a  b       0      0
   b       1      0

In [47]:
a = pd.DataFrame({
    'index': ['2000/01/01', '2001/01/01'],
    'col': [0, 1],
})
a['index'] = pd.to_datetime(a['index']).dt.date.astype(object)

b = pd.DataFrame({
    'index_date': ['2000/01/01', '2001/01/01'],
    'index_string': ['a', 'b'],
    'col': [0, 1],
})
b['index_date'] = pd.to_datetime(b['index_date']).dt.date.astype(object)

In [48]:
a.dtypes

index    object
col       int64
dtype: object

In [49]:
b.dtypes

index_date      object
index_string    object
col              int64
dtype: object

In [50]:
a = a.set_index(['index'])
b = b.set_index(['index_date', 'index_string'])

In [51]:
a.reset_index().dtypes

index    object
col       int64
dtype: object

In [52]:
b.reset_index().dtypes

index_date      datetime64[ns]
index_string            object
col                      int64
dtype: object